# Implementacja modelu uczenia maszynowego na potrzeby predykcji kursu indeksu WIG20

Ideą numer 1 jest próba wyłonienia najbardziej istotnych informacji, które zwiększają prawdopodobieństwo podjęcia właściwej decyzji inwestycyjnej.
Ideą numer 2 jest stworzenie modelu uczenia maszynowego, aby sprawdzić jego przydatność w tradingu. Używana do budowy modeli jest bibliteka scikit-learn.
Idea numer 3 to próba wyłonienia powiązań między wskaźnikami, które z wyprzedzeniem informują o zbliżających się sygnałach inwestycyjnych

### Instalacja niezbędnych bibliotek

Bazuję na danych historycznych pochodzących z serwisu: https://www.dukascopy.com/swiss/pl/marketwatch/historical/
Dukascopy daje możliwość pobierania danych za pomocą specjalnej biblioteki pythonowej, co jest bardzo wygodnym rozwiązaniem. Nazwa biblioteki: dukascopy-python

In [46]:
#pip install dukascopy-python

Biblioteka dukascopy-python pobiera dane prosto do struktury danych Pandas DataFrame, więc trzeba zainstalować biblitekę Pandas

In [47]:
#pip install pandas

Sprawdzenie możliwości biblioteki

Niezbędna będzie malipulacja datami: biblioteka datetime

In [48]:
#pip install datetime

Skoro ML to biblioteka scikit-learn

In [49]:
#pip install scikit-learn

Jakaś biblioteka do analizy technicznej

In [50]:
#pip install pandas-ta

### Import bibliotek

In [51]:
from datetime import datetime
#
import dukascopy_python as dcp
#
import pandas as pd
#
import pandas_ta as ta

Wyświetlenie dokumentacji biblioteki dukascopy_python

In [52]:
help(dcp)

Help on package dukascopy_python:

NAME
    dukascopy_python

PACKAGE CONTENTS
    instruments

FUNCTIONS
    fetch(instrument: str, interval: str, offer_side: str, start: datetime.datetime, end: datetime.datetime, max_retries: int = 7, limit: int = 30000, debug=False)

    live_fetch(instrument: str, interval_value: int, time_unit: str, offer_side: str, start: datetime.datetime, end: datetime.datetime, max_retries: int = 7, limit: int = 30000, debug=False)

    sleep(...)
        sleep(seconds)

        Delay execution for a given number of seconds.  The argument may be
        a floating-point number for subsecond precision.

DATA
    INTERVAL_DAY_1 = '1DAY'
    INTERVAL_HOUR_1 = '1HOUR'
    INTERVAL_HOUR_4 = '4HOUR'
    INTERVAL_MIN_1 = '1MIN'
    INTERVAL_MIN_10 = '10MIN'
    INTERVAL_MIN_15 = '15MIN'
    INTERVAL_MIN_30 = '30MIN'
    INTERVAL_MIN_5 = '5MIN'
    INTERVAL_MONTH_1 = '1MONTH'
    INTERVAL_SEC_1 = '1SEC'
    INTERVAL_SEC_10 = '10SEC'
    INTERVAL_SEC_30 = '30SEC'
    I

Sprawdzenie listy dostępnych instrumentów

In [53]:
import dukascopy_python.instruments as dcpi

In [54]:
help(dcpi)

Help on module dukascopy_python.instruments in dukascopy_python:

NAME
    dukascopy_python.instruments

DATA
    INSTRUMENT_AUSTRIA_ANDR_AT_EUR = 'ANDR.AT/EUR'
    INSTRUMENT_AUSTRIA_EBS_AT_EUR = 'EBS.AT/EUR'
    INSTRUMENT_AUSTRIA_POST_AT_EUR = 'POST.AT/EUR'
    INSTRUMENT_AUSTRIA_RBI_AT_EUR = 'RBI.AT/EUR'
    INSTRUMENT_AUSTRIA_TKA_AT_EUR = 'TKA.AT/EUR'
    INSTRUMENT_AUSTRIA_VER_AT_EUR = 'VER.AT/EUR'
    INSTRUMENT_AUSTRIA_VIG_AT_EUR = 'VIG.AT/EUR'
    INSTRUMENT_AUSTRIA_VOE_AT_EUR = 'VOE.AT/EUR'
    INSTRUMENT_AUSTRIA_WIE_AT_EUR = 'WIE.AT/EUR'
    INSTRUMENT_BELGIUM_ABI_BE_EUR = 'ABI.BE/EUR'
    INSTRUMENT_BELGIUM_AGS_BE_EUR = 'AGS.BE/EUR'
    INSTRUMENT_BELGIUM_BELG_BE_EUR = 'BELG.BE/EUR'
    INSTRUMENT_BELGIUM_KBC_BE_EUR = 'KBC.BE/EUR'
    INSTRUMENT_BELGIUM_SOLB_BE_EUR = 'SOLB.BE/EUR'
    INSTRUMENT_BELGIUM_UCB_BE_EUR = 'UCB.BE/EUR'
    INSTRUMENT_BELGIUM_UMI_BE_EUR = 'UMI.BE/EUR'
    INSTRUMENT_BND_CFD_BUND_TR_EUR = 'BUND.TR/EUR'
    INSTRUMENT_BND_CFD_UKGILT_TR_GBP = 'UKGILT.

Sprawdzenie możliwości biblioteki Pandas TA

In [55]:
help(ta)

Help on package pandas_ta:

NAME
    pandas_ta

PACKAGE CONTENTS
    candles (package)
    core
    custom
    cycles (package)
    momentum (package)
    overlap (package)
    performance (package)
    statistics (package)
    trend (package)
    utils (package)
    volatility (package)
    volume (package)

SUBMODULES
    alphavantage
    data
    yahoofinance

FUNCTIONS
    localtime(...)
        localtime([seconds]) -> (tm_year,tm_mon,tm_mday,tm_hour,tm_min,
                                  tm_sec,tm_wday,tm_yday,tm_isdst)

        Convert seconds since the Epoch to a time tuple expressing local time.
        When 'seconds' is not passed in, convert the current time instead.

    mfloor = floor(x, /)
        Return the floor of x as an Integral.

        This is the largest integer <= x.

    mul(a, b, /)
        Same as a * b.

    npArray = array(...)
        array(object, dtype=None, *, copy=True, order='K', subok=False, ndmin=0,
              like=None)

        Create an arra

Sprawdzenie jakie wskaźniki i funkcje są dostępne w bibliotece Pandas-TA

In [56]:
df = pd.DataFrame()
lista_wskaźników = [metoda for metoda in dir(df.ta) if not metoda.startswith('_')]

print(f"Znaleziono {len(lista_wskaźników)} wskaźników i funkcji.")
print(lista_wskaźników)

Znaleziono 157 wskaźników i funkcji.
['aberration', 'above', 'above_value', 'accbands', 'ad', 'adjusted', 'adosc', 'adx', 'alma', 'amat', 'ao', 'aobv', 'apo', 'aroon', 'atr', 'bbands', 'below', 'below_value', 'bias', 'bop', 'brar', 'categories', 'cci', 'cdl_pattern', 'cdl_z', 'cfo', 'cg', 'chop', 'cksp', 'cmf', 'cmo', 'constants', 'coppock', 'cores', 'cross', 'cross_value', 'cti', 'datetime_ordered', 'decay', 'decreasing', 'dema', 'dm', 'donchian', 'dpo', 'ebsw', 'efi', 'ema', 'entropy', 'eom', 'er', 'eri', 'exchange', 'fisher', 'fwma', 'ha', 'hilo', 'hl2', 'hlc3', 'hma', 'hwc', 'hwma', 'ichimoku', 'increasing', 'indicators', 'inertia', 'jma', 'kama', 'kc', 'kdj', 'kst', 'kurtosis', 'kvo', 'last_run', 'linreg', 'log_return', 'long_run', 'macd', 'mad', 'massi', 'mcgd', 'median', 'mfi', 'midpoint', 'midprice', 'mom', 'natr', 'nvi', 'obv', 'ohlc4', 'pdist', 'percent_return', 'pgo', 'ppo', 'psar', 'psl', 'pvi', 'pvo', 'pvol', 'pvr', 'pvt', 'pwma', 'qqe', 'qstick', 'quantile', 'reverse', 'r

### Definicja paramertów dla dla funkcji fetch() z biblioteki dukascopy_python

In [57]:
instrument = 'EUR/USD'
interval = '1DAY'
offer_side = 'B' 
start_date = datetime(2000, 1, 1)
end_date = datetime.today()

### Pobranie danych do Pandas DataFrame

In [58]:
df = dcp.fetch(instrument, interval, offer_side, start_date, end_date)

INFO:DUKASCRIPT:current timestamp :2025-07-30T02:00:00


In [59]:
df.head()

,open,high,low,close,volume
timestamp,,,,,
2003-05-04 00:00:00+00:00,1.12273,1.12338,1.12160,1.12169,257040.5
2003-05-05 00:00:00+00:00,1.12161,1.13009,1.12014,1.12924,1276005.6
2003-05-06 00:00:00+00:00,1.12921,1.14506,1.12723,1.14234,1281467.0
2003-05-07 00:00:00+00:00,1.14218,1.14323,1.13265,1.13494,1272596.6
2003-05-08 00:00:00+00:00,1.13507,1.15077,1.13006,1.14820,1266581.3


In [60]:
df.tail()

,open,high,low,close,volume
timestamp,,,,,
2025-07-25 00:00:00+00:00,1.17548,1.17609,1.17028,1.17396,84426.67
2025-07-27 00:00:00+00:00,1.17552,1.17686,1.17507,1.17554,4400.20
2025-07-28 00:00:00+00:00,1.17554,1.17700,1.15850,1.15963,94411.61
2025-07-29 00:00:00+00:00,1.15960,1.15989,1.15188,1.15506,106375.16
2025-07-30 00:00:00+00:00,1.15506,1.15725,1.14008,1.14278,154109.87


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7150 entries, 2003-05-04 00:00:00+00:00 to 2025-07-30 00:00:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    7150 non-null   float64
 1   high    7150 non-null   float64
 2   low     7150 non-null   float64
 3   close   7150 non-null   float64
 4   volume  7150 non-null   float64
dtypes: float64(5)
memory usage: 335.2 KB


Zbiór jest kompletny, zgodnie z oczekiwaniami zawiera dane do dnia poprzedzającego dzień uruchomienia skryptu.
Pora zatem na rozbudowanie zbioru i policzenie interesujących nas statystyk.

In [179]:
aberration
accbands
ad
adosc
adx
alma
amat
ao
aobv
apo
aroon
atr
bbands
bias
bop
brar
cci
cfo
cg
chop
cksp
cmf
cmo

cti
decay
dema
dm
donchian
dpo
ebsw, 
efi, 
ema, 
entropy, 
eom, 
er, 
eri
fisher
fwma
hilo
hma
hwc / wcp
hwma
ichimoku
inertia
jma
kama
kc
kdj
kst
kurtosis
kvo
linreg
log_return
macd
mad
massi
mcgd
median
mfi
midpoint
midprice
mom
natr
nvi
obv
ohlc4
pdist
percent_return
pgo
ppo
psar
psl
pvi
pvo
pvol
pvr
pvt
pwma
qqe
qstick
rma
roc
rsi
rsx
rvgi
rvi
sinwma
skew
slope
sma
smi
squeeze
squeeze_pro
ssf
stc
stdev
stoch
stochrsi
supertrend
swma
t3
td_seq
tema
thermo
tos_stdevall
trima
trix
true_range
tsi
ttm_trend
ui
uo
variance
vhf
vidya
vortex
vp
vwap
vwma 
willr
wma
zlma
zscore 

NameError: name 'aberration' is not defined

In [ ]:
cdl_pattern
cdl_z
coppock <- oscylator miesięczny, trzeba dostosować parametry: 14 -> 14*21=294, 11 -> 11*21=231, 10 -> 10*21=210
Default Inputs:
            length=10, fast=11, slow=14
quantile